In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import random
import collections
import time
import pickle
import re

In [2]:
hw_dir = '/home/yao/gm/mlds/final'

In [3]:
test_max_slen = 20
train_softmax_slen = 50
train_max_slen = 70

max_slen = 70
min_slen = 5

batch_size = 1280


# In[3]:

with open('%s/all_pickles'%hw_dir,'rb') as f1:
    [train_data,test_data,word2ix,ix2word,vocab_size] = pickle.load(f1)


# In[4]:

blank_ix = word2ix[' ']
new_data = []
for data in train_data:
    temp = []
    sc = 0
    if(int(data[0]) < 3):
        sc = 0
    elif(int(data[0]) > 3):
        sc = 1
    else:
        continue
    temp.append(sc)
    temp.append([int(x) for x in data[1].split(" ")])
    new_data.append(temp)
train_data = new_data
train_data = [i for i in train_data if len(i[1])<=70 and len(i[1])>=5]

blank_ix = word2ix[' ']
new_data = []
for data in test_data:
    temp = []
    sc = 0
    if(int(data[0]) < 3):
        sc = 0
    elif(int(data[0]) > 3):
        sc = 1
    else:
        continue
    temp.append(sc)
    temp.append([int(x) for x in data[1].split(" ")])
    new_data.append(temp)
test_data = new_data

len(train_data),len(test_data)


# In[5]:

train_data_dict = dict()
for i in train_data:
    len_cmmt = len(i[1])
    if len_cmmt in train_data_dict:
        train_data_dict[len_cmmt].append(i)
    else:
        train_data_dict[len_cmmt] = [i]

del train_data

# In[6]:

len_pos    = max_slen  # 現在在長度多少的file，從長的開始
batch_pos  = 0
def get_batch(batch_size):
    global train_data_dict, len_pos,batch_pos
    batch_size = (batch_size//len_pos)+1 # 越長的句子一個batch就讀越少句，+1是避免0的狀況
    # ====================回傳用的參數=============================
    is_eoi = False  # end of iteration
    is_eof = False
    slen  = len_pos
    # ===========================================================
    # ====================從檔案中讀batch_size句===================
    now_datas = train_data_dict[len_pos][batch_pos:batch_pos+batch_size]
    batch_pos+=batch_size
    if len(now_datas)<batch_size:
        batch_pos=0
        is_eof = True
    # ===========================================================
    batch_size = len(now_datas) # 因為有可能在檔案的尾巴 讀不滿完整的batch_size 故在此更新batch_size
    # ==================若現在的檔案讀完============================
    if is_eof == True:
        #random.shuffle(train_data_dict[len_pos])
        if len_pos == min_slen: # 一個iteration跑完 重新再跑
            len_pos = max_slen
            is_eoi = True
        else:                      # 現在這長度的file完了 開始讀len_pos-1的檔案
            len_pos -= 1
        if batch_size == 0: # 剛好讀到尾巴了
            return is_eoi,is_eof,None,None,slen,batch_size
    # ============================================================
        
    score = np.array([i[0] for i in now_datas])
    cmmt = np.array([i[1] for i in now_datas])
    return is_eoi,is_eof,score,cmmt,slen,batch_size

In [4]:
lr = 0.001                   
training_iters = 100      

n_vocab_size = vocab_size
n_filter = 64
n_filter_size = 4
n_classes = 2
n_max_pool = 2
n_hidden_units = 512
n_embed_size = 300
n_batch_size = 5000


# In[10]:

tf.reset_default_graph()


# In[11]:

word_embed = tf.Variable(tf.random_uniform([vocab_size,n_embed_size]))
y_embed = tf.constant(np.eye(n_classes))


# # CNN

# In[12]:

# n_sentence_length = tf.placeholder(tf.int32,[1])
x = tf.placeholder(tf.int32, [None, None])
# [2,4,3]
y = tf.placeholder(tf.int32, [None,])
ph_batch_size = tf.placeholder(tf.int32,[])
ph_sen_len = tf.placeholder(tf.int32,[])

# In[13]:

w_conv = tf.Variable(tf.truncated_normal([n_filter_size,n_embed_size,1,n_filter], stddev=0.1))
# [2,3,1,6]
b_conv = tf.Variable(tf.constant(0.1, shape=[n_filter]))

sen_index = x
x_emb = tf.nn.embedding_lookup(word_embed,sen_index)
y_index = y
y_emb = tf.nn.embedding_lookup(y_embed,y_index)

# In[14]:

x_4dim = tf.reshape(x_emb,[-1,ph_sen_len,n_embed_size,1])
# [2,4,3,1]


# In[15]:

h_conv = tf.nn.relu(tf.nn.conv2d(x_4dim, w_conv, strides=[1, 1, 1, 1], padding='VALID') + b_conv)
# [2,2,1,6]
h_pool = tf.nn.max_pool(h_conv, ksize=[1, 2, 1, 1], strides=[1, 2, 1, 1], padding='VALID')
# [2,1,1,6]


# In[16]:

RNN_input = tf.reshape(h_pool,[ph_batch_size,tf.shape(h_pool)[1],n_filter])


# # RNN

# In[17]:

inW = tf.Variable(tf.random_normal([n_filter, n_hidden_units]))
outW = tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
inB = tf.Variable(tf.constant(0.1, shape=[n_hidden_units, ]))
outB = tf.Variable(tf.constant(0.1, shape=[n_classes, ]))


# In[18]:

n_steps = tf.shape(h_pool)[1]
RNN_input1 = tf.reshape(RNN_input,[-1,n_filter])
X_in = tf.matmul(RNN_input1, inW) + inB
X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_units])
X_in = tf.nn.relu(X_in)
lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units, forget_bias=1.0, state_is_tuple=True)
init_state = lstm_cell.zero_state(ph_batch_size, dtype=tf.float32)
outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, X_in, initial_state=init_state, time_major=False)
outputs = tf.transpose(outputs, [1, 0, 2])[-1]
outputs1 = tf.reshape(outputs,[-1,n_hidden_units])
pred = tf.matmul(outputs1, outW) + outB
pred_argmax = tf.argmax(pred, 1)

# In[19]:

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_emb, logits=pred))
train_op = tf.train.AdamOptimizer(lr).minimize(cost)
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y_emb, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))    

In [5]:
sess = tf.Session()
saver = tf.train.Saver()
saver.restore(sess,'%s/ver2_model.bin'%hw_dir)


ans_list = []
is_eoi,is_eof,score,cmmt,slen,batch_size = get_batch(n_batch_size)
batch_now = 0
while(not is_eoi):
    batch_now += 1
    if batch_size == 0:
        is_eoi,is_eof,score,cmmt,slen,batch_size = get_batch(n_batch_size)
        continue
    inp = {x:cmmt,y:score,ph_batch_size:batch_size,ph_sen_len:slen}
    ans = sess.run(pred_argmax,inp)
    ans_list.append([cmmt,score,ans])
    is_eoi,is_eof,score,cmmt,slen,batch_size = get_batch(n_batch_size)

In [16]:
ans_list[0][2].tolist()

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [17]:
ans_list2 = []
for ans in ans_list:
    bsize = ans[0].shape[0]
    ans[0] = ans[0].tolist()
    ans[1] = ans[1].tolist()
    ans[2] = ans[2].tolist()
    for i in range(bsize):
        cmmt = ''.join([ix2word[j] for j in ans[0][i]])
        ans_list2.append([cmmt,ans[1][i],ans[2][i]])

In [21]:
yes = 0
yes_no = 0
for i in ans_list2:
    if i[1]==i[2]:
        yes+=1
    yes_no +=1

In [22]:
yes/float(yes_no)

0.5803024473972359

In [18]:
ans_list2[:100]

[['<B>28号app突然弹出到货消息 28号app突然弹出到货消息 立马下单 白条支付 刚巧遇上台风影响 本来应该上午就能到的结果等到下午才收到<E>',
  1,
  1],
 ['<B>总的来说还是很不错的 总的来说还是很不错的 首先指纹解锁的速度明显提升 系统运行更加流畅 其次 材质的硬度确实相比上一代有很大程度的提升<E>',
  1,
  1],
 ['<B>金属后盖手感非常好 4000毫安大电池就是爽 收到货是31 电量 用了40分钟才掉到29 这个价钱实在找不到比红米note3更合适的机型<E>',
  1,
  1],
 ['<B>一 桌面app图标不能随意确定位置 用了媳妇的照片当壁纸 还得都挤到文件夹里 不能遮住她脸了 二 我记得有多少名以内下单会有京东特殊包装<E>',
  1,
  1],
 ['<B>这点比iphone好 黑屏手势这功能真的很赞啊 手机5 5寸 考虑到单手操作不方便 很贴心的单手操作功能完美的解决了单手操作不方便的问题<E>',
  1,
  1],
 ['<B>好在华为的手机设置好后可以很方便隐藏和调出虚拟按键 还有个弊端就是没法把指纹识别放正面了 大屏幕手机把指纹识别放在背面有一定操作上的便利<E>',
  1,
  1],
 ['<B>那里买二四九九 在这里有的符件他们都有 而且还多了个什么精美礼品 我在这里还贵了一百大洋 而且他们的屏幕和我这个比他们的感觉上要清晰一点<E>',
  1,
  1],
 ['<B>非常满意 屏幕大 但感觉有点大 电池耐用 双摄像头成像好 夜间拍摄能力强 不过配套的充电线有点短 如果要体验vr的话话还是2k屏版本的好<E>',
  1,
  1],
 ['<B>后来申请换货 客服告诉我因为过了24小时没办法办理 很不理解 毕竟是那么细小的地方 如果不是朋友玩的时候发现自己都不知道什么时候才能发现<E>',
  0,
  1],
 ['<B>耳机有问题 耳机有问题 是正版的 屏幕凹凸不平 我贴钢化膜的是别人贴的第一板的时候两个边都贴不上去以为别人没贴好后来别人扯了再贴还是那样<E>',
  0,
  1],
 ['<B>耳机有问题 耳机有问题 是正版的 屏幕凹凸不平 我贴钢化膜的是别人贴的第一板的时候两个边都贴不上去以为别人没贴好后来别人扯了再贴还是那样<E>',
  0,
  1],

In [7]:
ans_list[0]

[array([[  11,    3,    9, ..., 1348,  344,   12],
        [  11, 1078, 1993, ..., 1301,  403,   12],
        [  11, 2857,  908, ..., 1470,  691,   12],
        ..., 
        [  11, 2074,  150, ...,   30,   30,   12],
        [  11, 1946, 1641, ..., 1939,  727,   12],
        [  11,   49, 2772, ..., 1074, 1993,   12]]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1,
        1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,
        1, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1])]